# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 18 2023 11:41AM,261327,19,9387542,MedStone Pharma LLC,Released
1,Apr 18 2023 11:33AM,261325,10,MSP220860,"Methapharm, Inc.",Released
2,Apr 18 2023 11:33AM,261325,10,MSP220861,"Methapharm, Inc.",Released
3,Apr 18 2023 11:33AM,261325,10,MSP220864,"Methapharm, Inc.",Released
4,Apr 18 2023 11:33AM,261325,10,MSP220865,"Methapharm, Inc.",Released
5,Apr 18 2023 11:33AM,261325,10,MSP220867,"Methapharm, Inc.",Released
6,Apr 18 2023 11:33AM,261325,10,MSP220869,"Methapharm, Inc.",Released
7,Apr 18 2023 11:33AM,261325,10,MSP220872,"Methapharm, Inc.",Released
8,Apr 18 2023 11:33AM,261325,10,MSP220873,"Methapharm, Inc.",Released
9,Apr 18 2023 11:33AM,261325,10,MSP220874,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,261319,Released,1
40,261322,Released,1
41,261324,Released,1
42,261325,Released,11
43,261327,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261319,NaN,NaN,1.0
261322,NaN,NaN,1.0
261324,NaN,NaN,1.0
261325,NaN,NaN,11.0
261327,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260872,5.0,34.0,14.0
261209,0.0,0.0,1.0
261225,0.0,3.0,27.0
261231,9.0,1.0,2.0
261234,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260872,5,34,14
261209,0,0,1
261225,0,3,27
261231,9,1,2
261234,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260872,5,34,14
1,261209,0,0,1
2,261225,0,3,27
3,261231,9,1,2
4,261234,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260872,5,34,14
1,261209,,,1
2,261225,,3,27
3,261231,9,1,2
4,261234,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 18 2023 11:41AM,261327,19,MedStone Pharma LLC
1,Apr 18 2023 11:33AM,261325,10,"Methapharm, Inc."
12,Apr 18 2023 11:29AM,261324,10,"Methapharm, Inc."
13,Apr 18 2023 11:22AM,261322,21,"NBTY Global, Inc."
14,Apr 18 2023 11:01AM,261318,10,Hush Hush
15,Apr 18 2023 11:00AM,261317,21,"NBTY Global, Inc."
16,Apr 18 2023 11:00AM,261316,19,"Methapharm, Inc."
20,Apr 18 2023 10:50AM,261314,19,"Emersa Waterbox, LLC"
22,Apr 18 2023 10:37AM,261311,10,"Uniderm USA, Inc - Collagenil Line"
23,Apr 18 2023 10:34AM,261304,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 18 2023 11:41AM,261327,19,MedStone Pharma LLC,,,1
1,Apr 18 2023 11:33AM,261325,10,"Methapharm, Inc.",,,11
2,Apr 18 2023 11:29AM,261324,10,"Methapharm, Inc.",,,1
3,Apr 18 2023 11:22AM,261322,21,"NBTY Global, Inc.",,,1
4,Apr 18 2023 11:01AM,261318,10,Hush Hush,,,1
5,Apr 18 2023 11:00AM,261317,21,"NBTY Global, Inc.",,,1
6,Apr 18 2023 11:00AM,261316,19,"Methapharm, Inc.",,3,1
7,Apr 18 2023 10:50AM,261314,19,"Emersa Waterbox, LLC",,,2
8,Apr 18 2023 10:37AM,261311,10,"Uniderm USA, Inc - Collagenil Line",,,1
9,Apr 18 2023 10:34AM,261304,10,Bio-PRF,,5,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 11:41AM,261327,19,MedStone Pharma LLC,1,,
1,Apr 18 2023 11:33AM,261325,10,"Methapharm, Inc.",11,,
2,Apr 18 2023 11:29AM,261324,10,"Methapharm, Inc.",1,,
3,Apr 18 2023 11:22AM,261322,21,"NBTY Global, Inc.",1,,
4,Apr 18 2023 11:01AM,261318,10,Hush Hush,1,,
5,Apr 18 2023 11:00AM,261317,21,"NBTY Global, Inc.",1,,
6,Apr 18 2023 11:00AM,261316,19,"Methapharm, Inc.",1,3,
7,Apr 18 2023 10:50AM,261314,19,"Emersa Waterbox, LLC",2,,
8,Apr 18 2023 10:37AM,261311,10,"Uniderm USA, Inc - Collagenil Line",1,,
9,Apr 18 2023 10:34AM,261304,10,Bio-PRF,6,5,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 11:41AM,261327,19,MedStone Pharma LLC,1,,
1,Apr 18 2023 11:33AM,261325,10,"Methapharm, Inc.",11,,
2,Apr 18 2023 11:29AM,261324,10,"Methapharm, Inc.",1,,
3,Apr 18 2023 11:22AM,261322,21,"NBTY Global, Inc.",1,,
4,Apr 18 2023 11:01AM,261318,10,Hush Hush,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 11:41AM,261327,19,MedStone Pharma LLC,1.0,NaN,NaN
1,Apr 18 2023 11:33AM,261325,10,"Methapharm, Inc.",11.0,NaN,NaN
2,Apr 18 2023 11:29AM,261324,10,"Methapharm, Inc.",1.0,NaN,NaN
3,Apr 18 2023 11:22AM,261322,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Apr 18 2023 11:01AM,261318,10,Hush Hush,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 11:41AM,261327,19,MedStone Pharma LLC,1.0,0.0,0.0
1,Apr 18 2023 11:33AM,261325,10,"Methapharm, Inc.",11.0,0.0,0.0
2,Apr 18 2023 11:29AM,261324,10,"Methapharm, Inc.",1.0,0.0,0.0
3,Apr 18 2023 11:22AM,261322,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Apr 18 2023 11:01AM,261318,10,Hush Hush,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3135591,51.0,9.0,0.0
12,261209,1.0,0.0,0.0
15,522544,28.0,3.0,0.0
16,261279,0.0,1.0,0.0
19,3135173,23.0,25.0,9.0
20,1044735,19.0,34.0,5.0
21,522639,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3135591,51.0,9.0,0.0
1,12,261209,1.0,0.0,0.0
2,15,522544,28.0,3.0,0.0
3,16,261279,0.0,1.0,0.0
4,19,3135173,23.0,25.0,9.0
5,20,1044735,19.0,34.0,5.0
6,21,522639,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,51.0,9.0,0.0
1,12,1.0,0.0,0.0
2,15,28.0,3.0,0.0
3,16,0.0,1.0,0.0
4,19,23.0,25.0,9.0
5,20,19.0,34.0,5.0
6,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,51.0
1,12,Released,1.0
2,15,Released,28.0
3,16,Released,0.0
4,19,Released,23.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,9.0,5.0,0.0
Executing,9.0,0.0,3.0,1.0,25.0,34.0,0.0
Released,51.0,1.0,28.0,0.0,23.0,19.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,9.0,5.0,0.0
1,Executing,9.0,0.0,3.0,1.0,25.0,34.0,0.0
2,Released,51.0,1.0,28.0,0.0,23.0,19.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,9.0,5.0,0.0
1,Executing,9.0,0.0,3.0,1.0,25.0,34.0,0.0
2,Released,51.0,1.0,28.0,0.0,23.0,19.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()